In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
DROP Table IF EXISTS Person;
DROP Table IF EXISTS Drug;
DROP Table IF EXISTS DrugUse;

In [3]:
CREATE TABLE Person (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100) NOT NULL,
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE DrugUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Person(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

In [4]:
SELECT * FROM Person;
SELECT * FROM Drug;
SELECT * FROM DrugUse

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT D.NAME, COUNT(*) NUM_PESSOAS
    FROM PERSON P, DRUG D, DRUGUSE U
    WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
    GROUP BY D.CODE

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
DROP TABLE IF EXISTS CONTAGEM_USO;

CREATE VIEW CONTAGEM_USO AS
SELECT D.CODE CODE, COUNT(*) NUM_PESSOAS
    FROM PERSON P, DRUG D, DRUGUSE U
    WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
    GROUP BY D.CODE;
    
SELECT CODE 
    FROM CONTAGEM_USO
    WHERE NUM_PESSOAS = (SELECT MAX(NUM_PESSOAS) FROM CONTAGEM_USO)

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
SELECT CODE 
    FROM (SELECT D.CODE CODE, COUNT(*) NUM_PESSOAS
        FROM PERSON P, DRUG D, DRUGUSE U
        WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
        GROUP BY D.CODE
        ORDER BY NUM_PESSOAS DESC)
    WHERE ROWNUM=1

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
SELECT NAME 
    FROM (SELECT D.NAME NAME, COUNT(*) NUM_PESSOAS
        FROM PERSON P, DRUG D, DRUGUSE U
        WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
        GROUP BY D.CODE
        ORDER BY NUM_PESSOAS DESC)
    WHERE ROWNUM=1

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [9]:
DROP TABLE IF EXISTS MEDICAMENTO_POR_PESSOA;

CREATE VIEW MEDICAMENTO_POR_PESSOA AS
SELECT P.ID, COUNT(*) NUM_MEDICAMENTOS
    FROM PERSON P, DRUG D, DRUGUSE U
    WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
    GROUP BY P.ID;
    
SELECT AVG(NUM_MEDICAMENTOS) FROM MEDICAMENTO_POR_PESSOA

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [10]:
DROP TABLE IF EXISTS MEDICAMENTO_POR_PESSOA;

CREATE VIEW MEDICAMENTO_POR_PESSOA AS
SELECT P.ID ID, COUNT(*) NUM_MEDICAMENTOS
    FROM PERSON P, DRUG D, DRUGUSE U
    WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
    GROUP BY P.ID;
    
SELECT ID, NUM_MEDICAMENTOS 
    FROM MEDICAMENTO_POR_PESSOA
    WHERE NUM_MEDICAMENTOS > (SELECT AVG(NUM_MEDICAMENTOS) FROM MEDICAMENTO_POR_PESSOA)

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [11]:
DROP TABLE IF EXISTS USO_ACIMA_DA_MEDIA;
DROP TABLE IF EXISTS MEDICAMENTO_POR_PESSOA;

CREATE VIEW MEDICAMENTO_POR_PESSOA AS
SELECT P.ID ID, P.AGE AGE, COUNT(*) NUM_MEDICAMENTOS
    FROM PERSON P, DRUG D, DRUGUSE U
    WHERE P.ID = U.PERSONID AND D.CODE=U.DRUGCODE
    GROUP BY P.ID;
    
SELECT AGE, AVG(NUM_MEDICAMENTOS) MEDIA_POR_IDADE
    FROM MEDICAMENTO_POR_PESSOA
    GROUP BY AGE

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo para `Marcadores` e `Taxonomia`:

![UML](marcadores-taxonomia-uml.png)

![Relacional](marcadores-taxonomia-er.png)

Considere as sentenças SQL a seguir para montar as tabelas de `Marcadores` e `Taxonomia`:

In [12]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

As tabelas são preenchidas com as seguintes tuplas:

In [13]:
INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Rede Social', 'Relacionamento');
INSERT INTO Taxonomia VALUES ('Marketplace', 'Vendas');
INSERT INTO Taxonomia VALUES ('Loja', 'Vendas');
INSERT INTO Taxonomia VALUES ('Notícias', 'Portal');
INSERT INTO Taxonomia VALUES ('Agregador', 'Portal');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Rede Social');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Amazon', 'https://www.amazon.com.br', 410, 'Marketplace');
INSERT INTO Marcadores VALUES ('Americanas', 'https://www.americanas.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Mercado Livre', 'https://www.mercadolivre.com.br', 500, 'Marketplace');
INSERT INTO Marcadores VALUES ('G1', 'https://g1.globo.com', 1200, 'Notícias');
INSERT INTO Marcadores VALUES ('Folha', 'https://www.folha.uol.com.br', 850, 'Notícias');
INSERT INTO Marcadores VALUES ('Google News', 'https://news.google.com', 900, 'Agregador');
INSERT INTO Marcadores VALUES ('Flipboard', 'https://flipboard.com', 950, 'Agregador');

Conteúdo da tabela `Taxonomia`:

In [14]:
SELECT * FROM Taxonomia;

Conteúdo da tabela `Marcadores`:

In [15]:
SELECT * FROM Marcadores;

## Tarefa 1

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, sem considerar as categorias subordinadas.

In [16]:
SELECT *
    FROM MARCADORES
    WHERE CATEGORIA='Serviços'

## Tarefa 2

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, considerando as categorias subordinadas.

In [17]:
DROP TABLE IF EXISTS SUB_SERVICOS3;
DROP TABLE IF EXISTS SUB_SERVICOS2;
DROP TABLE IF EXISTS SUB_SERVICOS;

In [18]:
/*Vamos construir uma tabela com a categoria 'Serviços' e todas as suas categorias subordinadas de primeiro grau*/
DROP TABLE IF EXISTS SUB_SERVICOS;

CREATE VIEW SUB_SERVICOS AS
SELECT CATEGORIA CAT1
    FROM TAXONOMIA
    WHERE CATEGORIA='Serviços' OR SUPERIOR='Serviços';
    
SELECT * FROM SUB_SERVICOS

In [19]:
/*Fazemos agora uma segunda tabela com subordinações de segundo grau, além das que já existem*/
DROP TABLE IF EXISTS SUB_SERVICOS2;

CREATE VIEW SUB_SERVICOS2 AS
SELECT DISTINCT T.CATEGORIA CAT2
    FROM TAXONOMIA T, SUB_SERVICOS S1
    WHERE T.CATEGORIA IN (S1.CAT1) OR T.SUPERIOR IN (S1.CAT1);
    
SELECT * FROM SUB_SERVICOS2

In [20]:
/*Verificamos se surgiram novas categorias*/
SELECT DISTINCT *
    FROM SUB_SERVICOS2
    WHERE CAT2 NOT IN (SELECT * FROM SUB_SERVICOS)

In [21]:
/*Como surgiram novas, vamos criar mais uma tabela com mais um grau de subordinação*/
DROP TABLE IF EXISTS SUB_SERVICOS3;

CREATE VIEW SUB_SERVICOS3 AS
SELECT DISTINCT T.CATEGORIA CAT3
    FROM TAXONOMIA T, SUB_SERVICOS S2
    WHERE T.CATEGORIA IN (S2.CAT1) OR T.SUPERIOR IN (S2.CAT1);
    
SELECT * FROM SUB_SERVICOS3

In [22]:
/*Verificamos novamente se surgiram novas categorias*/
SELECT DISTINCT *
    FROM SUB_SERVICOS3
    WHERE CAT3 NOT IN (SELECT * FROM SUB_SERVICOS2)

In [23]:
/*Como não surgiram novas categorias, então terminaram as classes subordinadas de serviços e utilizamos SUB_SERVICOS3 para fazer a consulta dos marcadores*/
SELECT *
    FROM MARCADORES
    WHERE CATEGORIA IN (SELECT * FROM SUB_SERVICOS3)